In [1]:
data_train = pd.read_csv("../fraud detection/input/creditcard_train.csv")
data_train.head()

NameError: name 'pd' is not defined

In [ ]:
data_test = pd.read_csv("../fraud detection/input/creditcard_test.csv")
data_test.head()

In [ ]:
print(data_train.shape)
print(data_test.shape)

In [ ]:
data_train.drop_duplicates(subset=None, inplace=True)
print(data_train.shape)

In [ ]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        dataframe[feature].hist(bins=20,ax=ax,facecolor='midnightblue')
        ax.set_title(feature+" Distribution",color='DarkRed')
        ax.set_yscale('log')
    fig.tight_layout()  
    plt.show()
draw_histograms(data_test, data_test.columns,8,4)

In [ ]:
data_train.columns

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(data_train['Time'])

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(data_train['Amount'])

In [ ]:
data_train.insert(30, 'Hour', data_train['Time'].apply(lambda x: np.ceil(float(x)/3600) % 24), allow_duplicates = True)
data_test.insert(30, 'Hour', data_test['Time'].apply(lambda x: np.ceil(float(x)/3600) % 24), allow_duplicates = True)

In [ ]:
pd.pivot_table(
    columns="Class", 
    index="Hour", 
    values= 'Amount', 
    aggfunc='count', 
    data=data_train)

In [ ]:
#Hour vs Class
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

sns.countplot(
    x="Hour",
    data=data_train[data_train['Class'] == 0], 
    color="#98D8D8",  
    ax=axes[0])
axes[0].set_title("Non-Fraudulent Transaction")


sns.countplot(
    x="Hour",
    data=data_train[data_train['Class'] == 1],
    color="#F08030", 
    ax=axes[1])
axes[1].set_title("Fraudulent Transaction")

In [ ]:
data_test.head()

In [ ]:
data_train.head()

In [ ]:
data_nonfraud = data_train[data_train['Class'] == 0].sample(2000)
data_fraud  = data_train[data_train['Class'] == 1]

data_new = data_nonfraud.append(data_fraud).sample(frac=1)
X = data_new.drop(['Class'], axis = 1).values
y = data_new['Class'].values

tsne = TSNE(n_components=2, random_state=42)
X_transformation = tsne.fit_transform(X)
#Y_transformation = tsne.fit_transform(data_test)

plt.figure(figsize=(10, 10))
plt.title("t-SNE Dimensionality Reduction")

def plot_data(X, y):
    plt.scatter(X[y == 0, 0], X[y == 0, 1], label="Non_Fraudulent", alpha=0.5, linewidth=0.15, c='#17becf')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], label="Fraudulent", alpha=0.5, linewidth=0.15, c='#d62728')
    plt.legend()
    return plt.show()

plot_data(X_transformation, y)

In [ ]:
data_train.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler().fit(data_train[['Time', 'Amount']])
data_train[['Time', 'Amount']] = Scaler.transform(data_train[['Time', 'Amount']])
data_test[['Time', 'Amount']] = Scaler.transform(data_test[['Time', 'Amount']])

In [ ]:
train_data, label_data = data_train.iloc[:,:-1],data_train.iloc[:,-1]
data_train.head()

In [ ]:
data_dmatrix = xgb.DMatrix(data=train_data, label= label_data)

X_train, X_test, y_train, y_test = train_test_split(
                                    train_data, label_data, test_size=0.3,random_state=42)
                                    
params = {
    'objective':'reg:logistic',
    'colsample_bytree': 0.3,
    'learning_rate': 0.1,
    'bootstrap': True, 
    'criterion': 'gini', 
    'max_depth': 4, 
    'max_features': 'auto', 
    'n_estimators': 50
}
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)

#Feature importance graph
plt.rcParams['figure.figsize'] = [20, 10]
xgb.plot_importance(xg_reg)

In [ ]:
data_model = data_train.drop(['V25', 'Time', 'V20', 'V22', 'V8', 'V2'], axis=1)
data_model_test = data_test.drop(['V25', 'Time', 'V20', 'V22', 'V8', 'V2'], axis=1)

In [ ]:
data_under_nonfraud = data_model[data_model['Class'] == 0].sample(15000)
data_under_fraud  = data_model[data_model['Class'] == 1]

data_undersampling = data_under_nonfraud.append(data_under_fraud, 
                                                ignore_index=True, sort=False)
                                                
plt.figure(figsize=(10,10))
sns.countplot(y="Class", data=data_undersampling,palette='Dark2')
plt.title('Fraudulent Transaction Summary')
plt.xlabel('Count')
plt.ylabel('Fraudulent Transaction,        Non-Fraudulent Transaction')

In [ ]:
data_test.info()

In [ ]:
modelLR = LogisticRegression(
    solver='lbfgs', 
    multi_class='multinomial',
    class_weight='balanced', 
    max_iter=500, 
    random_state=42
).fit(X_train, y_train)

# Obtain predictions from the test data 
predict_LR = modelLR.predict(X_test)
predict = modelLR.predict(data_test)

In [ ]:
data = {'Predicted':predict}
submissionDF = pd.DataFrame(data)
submissionDF.to_csv('submission_test.csv', index=True, index_label = "Id")